# Análisis exploratorio de Datos para correlación entre el consumo de café y el estado de salud de las personas

##### The GlobalCoffeeHealth dataset contains 10,000 synthetic records reflecting real-world patterns of coffee consumption, sleep behavior, and health outcomes across 20 countries. It includes demographics, daily coffee intake, caffeine levels, sleep duration and quality, BMI, heart rate, stress, physical activity, health issues, occupation, smoking, and alcohol consumption.

##### The dataset captures realistic correlations observed in research—such as caffeine’s impact on sleep, stress, and health—making it ideal for statistical analysis, predictive modeling, and lifestyle or wellness studies.

# Objetivo del estudio

### Objetivo general
##### Explorar la relación entre el consumo de café, los hábitos de vida y la salud, con el fin de identificar los principales indicadores de salud afectados, evaluar la interacción con otros factores de riesgo o protección, y determinar un rango de consumo de café que resulte más saludable para la población analizada.
### Objetivos específicos
##### Analizar qué indicadores de salud (sueño, calidad del sueño, IMC, frecuencia cardíaca, nivel de estrés) se ven más afectados por un alto consumo de café, y determinar la naturaleza de esas relaciones.
##### Explorar cómo factores asociados al estilo de vida, como el hábito de fumar, el consumo de alcohol y la actividad física, pueden potenciar o mitigar los efectos del consumo de café sobre la salud.
##### Estimar un nivel promedio de consumo de café que permita minimizar impactos negativos sobre indicadores de salud, identificando un rango considerado “seguro” o no perjudicial.

In [ ]:
import pandas as pd
import numpy as np

### Importando datos

In [ ]:
data = pd.read_csv('synthetic_coffee_health_10000.csv')
data.info()

In [ ]:
data.describe()

### Limpieza de base de datos

In [ ]:
print("Valores nulos por columna:\n", data.isnull().sum())
print("\nNúmero de duplicados:", data.duplicated().sum())

data.dtypes

### Transformaciones propuestas

In [ ]:
# Variables categóricas
categorical_vars = ["Gender", "Country", "Sleep_Quality", 
                    "Stress_Level", "Health_Issues", "Occupation"]
for col in categorical_vars:
    data[col] = data[col].astype("category")

# Variables binarias
binary_vars = ["Smoking", "Alcohol_Consumption"]
for col in binary_vars:
    data[col] = data[col].astype(int)

# Clasificación BMI
def classify_bmi(bmi):
    if bmi < 18.5:
        return "Underweight"
    elif 18.5 <= bmi < 25:
        return "Normal"
    elif 25 <= bmi < 30:
        return "Overweight"
    else:
        return "Obese"

data["BMI_Category"] = data["BMI"].apply(classify_bmi).astype("category")

# Vista previa después de transformaciones
data.head()